## Assignment 2: MongoDB

This assignment is based on content discussed in the Introduction to MongoDB module.

## Learning outcomes 

The purpose of this assignment is for learners to be able to:
- Familarize with JSON document syntax
- Understand basic MongoDB CRUD operations
- Understand MongoDB data pipelines to run aggregate queries

In this assignment, you will make use of the sample data provided.  

This dataset has 3 collections: Employee, Workplace and Address.  You will import this data into your local MongoDB database.

Required imports for this project are given below. Make sure you have all libraries required for this project installed. You may use conda or pip based on your set up.

## Setup Notes

**Please note** These instructions are duplicated here for your reference. You would have encountered them in a previous module. It is expected that you have already taken the steps to set up and run MongoDB.

We will be using MongoDB Community Edition.  The MongoDB database <b>MUST</b> be installed and running locally before continuing with this notebook.  We will need to install two packages using the Anaconda package manager:

1. [Mongodb](https://www.mongodb.com/) - this package contains the mongodb database 
2. [PyMongo](https://www.mongodb.com/docs/drivers/pymongo/) - this package contains the python driver that will allow us to communicate with the mongodb database.

#### Install

1. Open a command line terminal and execute the following command to install mongodb.
```console
conda install -c anaconda mongodb
```
2. Open a command line terminal and execute the following command to install pymongo packge.
```console
conda install -c anaconda pymongo
```

#### Run Mongodb in Windows
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is \data\db. Create this folder using the following commands from a Command Prompt:
```console
md \data\db
```

2. To start MongoDB, run mongod.exe. For example, from the Command Prompt:
```console
"C:\Program Files\MongoDB\Server\3.2\bin\mongod.exe"
```

#### Run Mongodb in Mac
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is /data/db. Create this folder using the following commands from a Command Prompt.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mkdir /data/db/
```

2. To start MongoDB, run mongod.exe. For example, from the Command Promp.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mongod --config /usr/local/etc/mongod.conf
```

In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
#required imports
import os
import json
import datetime
import pymongo
import pprint
import pandas as pd
import numpy as np
from pymongo import MongoClient
print('Mongo version', pymongo.__version__)

Mongo version 4.8.0


We first need to connect to our locally running MongoDB database (<b>Make sure your database is running on your machine</b>). We will use the MongoClient to connect to a local 'test' database that is running on port 27017 (this is the default port).

In [5]:
client = MongoClient('localhost', 27017)
db = client.assignment1

After installing necessary modules proceed to import the data into your database.

In [6]:
# Let's delete any existing collections in our database
db.workplace.drop()
db.address.drop()
db.employee.drop()

# Import our files into our three collections
with open('Data/Employee.json') as f:
    db.employee.insert_many(json.load(f))
with open('Data/Workplace.json') as f:
    db.workplace.insert_many(json.load(f))
with open('Data/Address.json') as f:
    db.address.insert_many(json.load(f))

#### Question 1 (10 Marks)

The address collection contains employee from different ages and interests.  Perform a simple query to list all employees that are less than or equal to 50 and like Cooking.

__NOTE:__ the following shows the structure of an Employee document that will help you construct the query.

In [7]:
pprint.pprint(client.assignment1.employee.find_one())

{'_id': '9f39da36-82cc-4353-ab90-d616105fa7c1',
 'address_id': 'b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8',
 'age': 40,
 'email': 'ih@ri.ro',
 'firstname': 'Emilie',
 'interests': ['Bowling', 'Cooking', 'Golf', 'Swimming'],
 'lastname': 'Woods',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}


In [8]:
count = db.employee.count_documents({})
print(f"Total number of employees: {count}")

Total number of employees: 100


In [9]:
# YOUR CODE HERE
#list all employees that are less than or equal to 50 and like Cooking
#the firstname, lastname, age, and interests columns are displayed and id column is not displayed.
cursor= db.employee.find(
    {"age": {"$lte":50}, 
    "interests" : "Cooking"}, 
    {"firstname": 1, "lastname": 1, "age": 1, "interests": 1, "_id":0})
#display results in a dataframe 
df= pd.DataFrame(list(cursor))
display(df)



,firstname,lastname,age,interests
0,Emilie,Woods,40,"[Bowling, Cooking, Golf, Swimming]"
1,Thomas,Patterson,18,"[Cooking, Cricket, Tennis, Swimming, Fishing]"
2,Sophia,Flores,22,"[Hiking, Soccer, Bowling, Rubgy, Cooking, Danc..."
3,Ollie,Barnett,25,"[Cooking, Bowling, Dancing]"
4,James,Wilkins,27,"[Rubgy, Tennis, Cricket, Cooking]"
5,Aaron,Carr,25,[Cooking]
6,Alta,Sharp,34,"[Cricket, Cycling, Rubgy, Golf, Cooking, Dancing]"
7,Delia,Douglas,36,"[Cricket, Cooking, Hiking, Dancing, Tennis]"
8,Dominic,Wade,48,"[Cycling, Dancing, Cooking]"
9,Myrtle,Little,36,"[Cooking, Cycling, Hiking, Rubgy, Bowling, Dan..."


#### Question 2  (10 Marks)

Insert a new Employee with the following properties:

* First Name: Jake 
* Last Name: Sample
* Email: jakesample@email.com
* Age: 26
* Interest: Biking, Hiking

Also, this employee works for 'Union Planters Corp' and lives at '573 Wojhas Square, Victoria'.
Verify that the insert succeeded and display the generated employees _id attribute.

__HINT__ An Employee document references a Workplace and Address document

In [10]:
cursor=db.address.find({})
df=pd.DataFrame(list(cursor))
display(df)

,_id,address,postalcode,city,province
0,91b5b7b3-2309-4e8a-8247-cd66d626ef0c,573 Wojhas Square,A7D 5A3,Victoria,BC
1,904568c4-3eb2-488d-90a1-37c40d2d182a,1630 Nozjal Park,A3Z 1F1,Markham,ON
2,91b32548-65b6-4fd9-91a1-2d86b04b8eea,1268 Vukani Parkway,X6J 7Y4,Calgary,AB
3,b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8,1594 Otefi Lane,S7T 9A5,St. John,NF
4,7e02bce2-d77c-4fd0-9794-3a53e99c06f5,584 Ducca Court,C8V 1U6,Vancouver,BC
...,...,...,...,...,...
155,5fd4089a-ba1f-482d-b5e3-6b05cd4bab02,84 Hepi Manor,E1O 5L5,Markham,ON
156,19728362-40ea-4f68-a072-0c44a912be12,671 Kowsi Park,V6N 9K3,Montreal,QB
157,be2aed2f-09ab-4b58-89ed-71f2155a20d6,711 Masule Junction,V8L 8W1,Brampton,ON
158,ee89a38b-24be-48d9-a072-8e80842ca08b,31 Enboc Ridge,A0L 9W7,Victoria,BC


In [11]:
cursor=db.workplace.find({})
df= pd.DataFrame(list(cursor))
display(df)

,_id,name,website,industry,address_id
0,5345fcb9-6297-4b9f-aa15-cbee8460f28f,Union Planters Corp,http://www.unionplanternscorp.com,Aerospace,9949fe3b-99ec-4485-b91d-823925db7d28
1,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f,Aetna Inc.,http://aetnainc.comaetnainc.com,Agriculture,70bbc861-703c-4501-90a6-c818a6372430
2,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Net Inc.,http://www.healthnetinc.com,Construction,1ed298fc-20ab-4750-ac38-fed1e60964af
3,b2903a4b-5688-4597-90ed-0f06d944bb6d,Kemet Corp.,http://www.kemetcorp.com,Education,45d25888-af94-488a-823f-40fdfc300696
4,b2a2844d-aeab-4602-b74c-01bf3b8e9c78,Bell Microproducts Inc.,http://www.bellmicroproductsinc.com,Finance,bc3790a7-b93f-43c0-85d8-73aa5a6b6f11
5,b12cd444-e65b-4bc2-8cf6-2dbe854a627b,Equity Residential Properties Trust,http://www.equityresidentialpropertiestrust.com,Agriculture,34886f23-8be7-4f7a-ba53-84fd1ca6e1bf
6,023acbac-b32a-477f-a65a-db11bd7d40f3,Equity Office Properties Trust,http://www.equityofficeproperties.com,Finance,a31241a6-b3c0-45c6-92ac-6c6513d06fac
7,cb795afb-8dc3-482f-b3a0-14229a280afa,Hilton Solutions,http://www.hiltonsolutions.com,Education,16e84458-c7ef-4fba-9c4b-058ae8a4bc71
8,a32bf18d-e0e5-48f2-a851-aa49c80f9460,Great Plains Energy Inc.,http://www.greatplainsenergy.com,Agriculture,c35b4c7f-b7de-431d-bd60-4026490cd61c
9,50275ad1-8140-4e79-8818-21793e3eb0a3,Xcel Bear Inc,http://www.xcelenergyinc.com,Construction,ed20462a-f836-476f-8b0c-49bb3abe2235


In [12]:
# YOUR CODE HERE #ANSWER

address_id= "91b5b7b3-2309-4e8a-8247-cd66d626ef0c"
workplace_id="5345fcb9-6297-4b9f-aa15-cbee8460f28f"

employee_doc= {"firstname": "Jake" , "lastname": "Sample", "email": "jakesample@email.com", "age":26, "interests": ['Biking' , 'Hiking'], "address_id": address_id,"workplace_id": workplace_id}
employee_id = db.employee.insert_one(employee_doc).inserted_id


#verify insert
print(f"Employee ID: {employee_id}")


Employee ID: 67aa68e494891e447941bd29


In [13]:
#Verify Result
pprint.pprint(client.assignment1.employee.find_one({'email':'jakesample@email.com'}))

{'_id': ObjectId('67aa68e494891e447941bd29'),
 'address_id': '91b5b7b3-2309-4e8a-8247-cd66d626ef0c',
 'age': 26,
 'email': 'jakesample@email.com',
 'firstname': 'Jake',
 'interests': ['Biking', 'Hiking'],
 'lastname': 'Sample',
 'workplace_id': '5345fcb9-6297-4b9f-aa15-cbee8460f28f'}


In [14]:
#Verify Result
cursor=db.employee.find({"lastname" : "Sample"})
df= pd.DataFrame(list(cursor))
display(df)

,_id,firstname,lastname,email,age,interests,address_id,workplace_id
0,67aa68e494891e447941bd29,Jake,Sample,jakesample@email.com,26,"[Biking, Hiking]",91b5b7b3-2309-4e8a-8247-cd66d626ef0c,5345fcb9-6297-4b9f-aa15-cbee8460f28f


In [15]:
cursor=db.address.find({"address" : "573 Wojhas Square"})
df= pd.DataFrame(list(cursor))
display(df)

,_id,address,postalcode,city,province
0,91b5b7b3-2309-4e8a-8247-cd66d626ef0c,573 Wojhas Square,A7D 5A3,Victoria,BC


In [16]:
cursor=db.workplace.find({"name" : "Union Planters Corp"})
df= pd.DataFrame(list(cursor))
display(df)

,_id,name,website,industry,address_id
0,5345fcb9-6297-4b9f-aa15-cbee8460f28f,Union Planters Corp,http://www.unionplanternscorp.com,Aerospace,9949fe3b-99ec-4485-b91d-823925db7d28


#### Question 3 (10 Marks)

Delete all employees that work for 'Great Plains Energy Inc.' and are greater than 46 years old and likes 'Tennis'. Once you delete the employees verify the number of employees deleted.

In [17]:
#EDA
df= pd.DataFrame(list(db.employee.find()))
display(df)

,_id,firstname,lastname,age,email,interests,address_id,workplace_id
0,9f39da36-82cc-4353-ab90-d616105fa7c1,Emilie,Woods,40,ih@ri.ro,"[Bowling, Cooking, Golf, Swimming]",b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8,a32bf18d-e0e5-48f2-a851-aa49c80f9460
1,077e6e5e-0d73-4b14-95a5-5d0d697a1ad8,Max,Craig,50,nazceg@esfazpup.mv,"[Cricket, Swimming]",7e02bce2-d77c-4fd0-9794-3a53e99c06f5,b12cd444-e65b-4bc2-8cf6-2dbe854a627b
2,af27265e-6639-49f2-991e-193275a4111a,Thomas,Patterson,18,sug@gon.bf,"[Cooking, Cricket, Tennis, Swimming, Fishing]",64fd714d-e219-4e45-888b-cc2238a8bd0b,5345fcb9-6297-4b9f-aa15-cbee8460f28f
3,f44811ee-0187-427f-b597-d44cc454c91f,Isabella,Cunningham,47,zunoj@itwi.ar,"[Rubgy, Tennis, Dancing]",4cf71d6d-4a7e-4731-a868-6ad03faf1f22,b12cd444-e65b-4bc2-8cf6-2dbe854a627b
4,c98c2311-6027-423d-9374-d1e2c2479828,Carl,Lambert,30,ozsam@razir.md,"[Rubgy, Dancing]",f72ee55d-8567-4fd7-a735-001e4ec00051,b2903a4b-5688-4597-90ed-0f06d944bb6d
...,...,...,...,...,...,...,...,...
96,220eb787-0a63-4518-9b3e-4f5c0afd68c3,Eula,Santos,55,zegatike@iwocejnan.sy,"[Rubgy, Cycling, Fishing]",c923b5b1-3eca-4a26-9974-cfadae5c1ddf,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f
97,b2a2ae86-f4f6-4da5-ade6-ee1cd135daf3,Gavin,Conner,49,ah@jopah.uz,"[Rubgy, Cooking, Bowling]",ef5a17a2-6f11-49b8-a0fd-e0bddc56f185,50275ad1-8140-4e79-8818-21793e3eb0a3
98,2bfc0479-cb15-46e1-94fa-801404791b21,Peter,Jimenez,37,usope@tega.bh,"[Cooking, Swimming, Cricket]",77ce0c50-afd6-47e8-86d7-b1b372e8deb2,cb795afb-8dc3-482f-b3a0-14229a280afa
99,663e70d1-1b08-4e40-a880-befe2bec42d3,Edward,Hawkins,52,bibtar@ru.gb,"[Hiking, Soccer, Dancing]",824fea9d-d0e6-4c53-ad9d-5bf626a948f3,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f


In [18]:
#EDA
df= pd.DataFrame(list(db.address.find()))
display(df)

,_id,address,postalcode,city,province
0,91b5b7b3-2309-4e8a-8247-cd66d626ef0c,573 Wojhas Square,A7D 5A3,Victoria,BC
1,904568c4-3eb2-488d-90a1-37c40d2d182a,1630 Nozjal Park,A3Z 1F1,Markham,ON
2,91b32548-65b6-4fd9-91a1-2d86b04b8eea,1268 Vukani Parkway,X6J 7Y4,Calgary,AB
3,b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8,1594 Otefi Lane,S7T 9A5,St. John,NF
4,7e02bce2-d77c-4fd0-9794-3a53e99c06f5,584 Ducca Court,C8V 1U6,Vancouver,BC
...,...,...,...,...,...
155,5fd4089a-ba1f-482d-b5e3-6b05cd4bab02,84 Hepi Manor,E1O 5L5,Markham,ON
156,19728362-40ea-4f68-a072-0c44a912be12,671 Kowsi Park,V6N 9K3,Montreal,QB
157,be2aed2f-09ab-4b58-89ed-71f2155a20d6,711 Masule Junction,V8L 8W1,Brampton,ON
158,ee89a38b-24be-48d9-a072-8e80842ca08b,31 Enboc Ridge,A0L 9W7,Victoria,BC


In [19]:
#EDA
df= pd.DataFrame(list(db.workplace.find()))
display(df)

,_id,name,website,industry,address_id
0,5345fcb9-6297-4b9f-aa15-cbee8460f28f,Union Planters Corp,http://www.unionplanternscorp.com,Aerospace,9949fe3b-99ec-4485-b91d-823925db7d28
1,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f,Aetna Inc.,http://aetnainc.comaetnainc.com,Agriculture,70bbc861-703c-4501-90a6-c818a6372430
2,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Net Inc.,http://www.healthnetinc.com,Construction,1ed298fc-20ab-4750-ac38-fed1e60964af
3,b2903a4b-5688-4597-90ed-0f06d944bb6d,Kemet Corp.,http://www.kemetcorp.com,Education,45d25888-af94-488a-823f-40fdfc300696
4,b2a2844d-aeab-4602-b74c-01bf3b8e9c78,Bell Microproducts Inc.,http://www.bellmicroproductsinc.com,Finance,bc3790a7-b93f-43c0-85d8-73aa5a6b6f11
5,b12cd444-e65b-4bc2-8cf6-2dbe854a627b,Equity Residential Properties Trust,http://www.equityresidentialpropertiestrust.com,Agriculture,34886f23-8be7-4f7a-ba53-84fd1ca6e1bf
6,023acbac-b32a-477f-a65a-db11bd7d40f3,Equity Office Properties Trust,http://www.equityofficeproperties.com,Finance,a31241a6-b3c0-45c6-92ac-6c6513d06fac
7,cb795afb-8dc3-482f-b3a0-14229a280afa,Hilton Solutions,http://www.hiltonsolutions.com,Education,16e84458-c7ef-4fba-9c4b-058ae8a4bc71
8,a32bf18d-e0e5-48f2-a851-aa49c80f9460,Great Plains Energy Inc.,http://www.greatplainsenergy.com,Agriculture,c35b4c7f-b7de-431d-bd60-4026490cd61c
9,50275ad1-8140-4e79-8818-21793e3eb0a3,Xcel Bear Inc,http://www.xcelenergyinc.com,Construction,ed20462a-f836-476f-8b0c-49bb3abe2235


In [20]:
#list all employees who are greater than 46 years old and like tennis and work for 'Great Plains Energy Inc.' work_id: a32bf18d-e0e5-48f2-a851-aa49c80f9460
cursor= db.employee.find(
    {"age": {"$gt":46}, 
    "interests" : "Tennis",
    "workplace_id": "a32bf18d-e0e5-48f2-a851-aa49c80f9460"}, 
    {"firstname": 1, "lastname": 1, "age": 1, "interests": 1, "_id":1, 'workplace_id': 1, 'address_id': 1})
#display results in a dataframe 
df= pd.DataFrame(list(cursor))
display(df)



,_id,firstname,lastname,age,interests,address_id,workplace_id
0,153ba346-b33d-4611-802c-fd95c7cb078c,Adelaide,Curry,71,[Tennis],8264af1e-1c55-4ed8-96f9-e3b9e1eff7d5,a32bf18d-e0e5-48f2-a851-aa49c80f9460
1,8e082f05-13b7-485d-89c4-259f1103f95c,Lester,Summers,48,"[Bowling, Tennis]",62996de2-fcda-4894-95d5-fef0d11d92f4,a32bf18d-e0e5-48f2-a851-aa49c80f9460
2,dfce1d98-15f6-4b75-86f6-cd5d05cfdc04,Sadie,Park,80,"[Golf, Fishing, Bowling, Dancing, Cooking, Ten...",2987cea4-b71d-4338-b0ef-a9b5feb3a138,a32bf18d-e0e5-48f2-a851-aa49c80f9460
3,86417f71-18cd-41c5-ad01-24b31ff2074f,Lucy,May,65,[Tennis],fe80c95a-a0e9-42c9-aa6a-56f27020842d,a32bf18d-e0e5-48f2-a851-aa49c80f9460


In [21]:
#NUMBER OF EMPLOYEES WHO ARE GREATER THAN 46, ARE INTERESTED IN TENNIS, AND WORK FOR GREAT PLAINS ENERGY INC.


db.employee.count_documents({
    "workplace_id": "a32bf18d-e0e5-48f2-a851-aa49c80f9460",
    "age": { "$gt": 46 },
    "interests": "Tennis"
})

4

In [22]:
# YOUR CODE HERE #ANSWER TO QUESTION#2
#delete
delete_result= db.employee.delete_many(
    {"age": {"$gt":46}, 
    "interests" : "Tennis",
    "workplace_id": "a32bf18d-e0e5-48f2-a851-aa49c80f9460"})


In [23]:
print("Number of employees deleted:", delete_result.deleted_count)

Number of employees deleted: 4


In [24]:
#now perform the above query
cursor= db.employee.find(
    {"age": {"$gt":46}, 
    "interests" : "Tennis",
    "workplace_id": "a32bf18d-e0e5-48f2-a851-aa49c80f9460"}, 
    {"firstname": 1, "lastname": 1, "age": 1, "interests": 1, "_id":1, 'workplace_id': 1, 'address_id': 1})
#display results in a dataframe 
df= pd.DataFrame(list(cursor))
display(df)
#THIS CODE VERIFIES THAT EMPLOYEES WHO ARE GREATER THAN 46 IN AGE, WORK FOR GREAT PLAINS ENERGY INC, AND ARE INTERESTED IN TENNIS HAVE BEEN DELETED SUCCESSFULLY. 


""


#### Question 4 (12 Marks)
Add a new field called 'industry' to all employees that work for 'Health Net Inc.' and populate the field with the value 'Health Care'.

__HINT__ Adding a new field to a document is like updating the document

In [25]:
# YOUR CODE HERE
#ANSWER TO QUESTION # 4

update_result= db.employee.update_many(
  { 'workplace_id': "a222385c-342c-43ea-adbc-9b487a2ee2be" }, 
  { '$set': { 'industry': "Health Care" } }
)

#verify result by viewing employees document and finding all employees with 'workplace_id' eqaul to a222385c-342c-43ea-adbc-9b487a2ee2be for 'Health Net Inc.' and see if the industry is set to Health Care
cursor=db.employee.find (
  {"workplace_id": "a222385c-342c-43ea-adbc-9b487a2ee2be"},
  {"firstname": 1, "lastname": 1, "age": 1, "interests": 1, "_id":1, 'workplace_id': 1, 'address_id': 1, 'industry': 1}
)

df=pd.DataFrame (list(cursor))
display(df)

,_id,firstname,lastname,age,interests,address_id,workplace_id,industry
0,ad4fd8bd-b732-4388-8469-5395b497df63,Marc,Castro,42,"[Golf, Cricket, Rubgy, Hiking, Fishing, Dancing]",91b32548-65b6-4fd9-91a1-2d86b04b8eea,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care
1,6222b4f5-6fd3-4c72-b058-710fb568b869,Lucinda,Gomez,46,"[Golf, Rubgy, Tennis]",59516270-19f4-47e8-95ae-3e83fa5f230c,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care
2,ca189ae9-d04b-4ae0-9b4a-b725196c31d2,Emilie,Robertson,37,[Hiking],16ff1778-b687-42ef-aace-36cedc05f521,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care
3,ed499274-f17f-4a4c-8327-7e740e6dd9c1,Josephine,Mills,36,"[Tennis, Bowling, Cricket]",01185f9b-b00f-4f2d-9878-0c989fe00fbc,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care
4,6a3bd505-56e6-46ec-bef1-2eef1fd3643c,Vernon,Edwards,24,[Soccer],a8f07070-cfb0-483e-a07a-b2adf9e3d4b0,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care
5,5b8bb404-5741-46a9-84f2-81248ffd2840,Glen,Turner,43,"[Golf, Hiking]",fc2b6b58-8392-4f3d-b1c8-07f4df87dc24,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care
6,c7cf8650-f391-4f2b-bbb5-60225729e998,Jeremiah,Payne,49,"[Cricket, Fishing]",44cc24b9-1d60-45cf-be0d-a225f35d52ab,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care
7,9ebcf9fc-e193-4ab7-9c38-10c2d6bf6036,Alan,Berry,76,"[Bowling, Cooking, Rubgy, Fishing, Golf, Cricket]",d34f9d8c-c1a0-4653-8a44-d2fe65714956,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care
8,fd677b4b-6eef-4777-bc3a-25d9a86134fa,Mattie,Fisher,37,"[Cycling, Fishing, Soccer, Rubgy, Hiking]",98fa15f0-5b73-4fb9-a5a4-1332e0f5b58b,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care
9,cb369c19-4bbc-449f-bff6-b198439d0d7b,Adele,Benson,43,"[Swimming, Hiking, Tennis]",d5b7473d-c82b-4956-bce3-80497ce6c015,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Care


In [26]:
cursor=db.employee.find ()

df=pd.DataFrame (list(cursor))
display(df)

,_id,firstname,lastname,age,email,interests,address_id,workplace_id,industry
0,9f39da36-82cc-4353-ab90-d616105fa7c1,Emilie,Woods,40,ih@ri.ro,"[Bowling, Cooking, Golf, Swimming]",b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8,a32bf18d-e0e5-48f2-a851-aa49c80f9460,NaN
1,077e6e5e-0d73-4b14-95a5-5d0d697a1ad8,Max,Craig,50,nazceg@esfazpup.mv,"[Cricket, Swimming]",7e02bce2-d77c-4fd0-9794-3a53e99c06f5,b12cd444-e65b-4bc2-8cf6-2dbe854a627b,NaN
2,af27265e-6639-49f2-991e-193275a4111a,Thomas,Patterson,18,sug@gon.bf,"[Cooking, Cricket, Tennis, Swimming, Fishing]",64fd714d-e219-4e45-888b-cc2238a8bd0b,5345fcb9-6297-4b9f-aa15-cbee8460f28f,NaN
3,f44811ee-0187-427f-b597-d44cc454c91f,Isabella,Cunningham,47,zunoj@itwi.ar,"[Rubgy, Tennis, Dancing]",4cf71d6d-4a7e-4731-a868-6ad03faf1f22,b12cd444-e65b-4bc2-8cf6-2dbe854a627b,NaN
4,c98c2311-6027-423d-9374-d1e2c2479828,Carl,Lambert,30,ozsam@razir.md,"[Rubgy, Dancing]",f72ee55d-8567-4fd7-a735-001e4ec00051,b2903a4b-5688-4597-90ed-0f06d944bb6d,NaN
...,...,...,...,...,...,...,...,...,...
92,220eb787-0a63-4518-9b3e-4f5c0afd68c3,Eula,Santos,55,zegatike@iwocejnan.sy,"[Rubgy, Cycling, Fishing]",c923b5b1-3eca-4a26-9974-cfadae5c1ddf,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f,NaN
93,b2a2ae86-f4f6-4da5-ade6-ee1cd135daf3,Gavin,Conner,49,ah@jopah.uz,"[Rubgy, Cooking, Bowling]",ef5a17a2-6f11-49b8-a0fd-e0bddc56f185,50275ad1-8140-4e79-8818-21793e3eb0a3,NaN
94,2bfc0479-cb15-46e1-94fa-801404791b21,Peter,Jimenez,37,usope@tega.bh,"[Cooking, Swimming, Cricket]",77ce0c50-afd6-47e8-86d7-b1b372e8deb2,cb795afb-8dc3-482f-b3a0-14229a280afa,NaN
95,663e70d1-1b08-4e40-a880-befe2bec42d3,Edward,Hawkins,52,bibtar@ru.gb,"[Hiking, Soccer, Dancing]",824fea9d-d0e6-4c53-ad9d-5bf626a948f3,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f,NaN


#### Question 5 (10 Marks)

Create an aggregate query to count the number of employees for each company and sort the output from largest employee count to lowest employee count.

__NOTE__ you will use a pipeline to achieve the computed result.  You should produce a result similar to the following table (the following table contains fake data)
<table>
    <tr><th></th><th>_id</th><th>count</th></tr>
    <tr><td>0</td><td>[Equity Residential Properties Trust]</td><td>19</td></tr>
    <tr><td>...</td><td>...</td><td>...</td></tr>
    <tr><td>7</td><td>[Bell Microproducts Inc.]</td><td>6</td></tr>
    <tr><td>8</td><td>[Kemet Corp.]</td><td>1</td></tr>
</table>

__HINT__ you should make use of the \\$lookup, \\$group and \\$sort pipeline operations

In [27]:
# YOUR CODE HERE
#ANSWER FOR QUESTION 5
query_result = db.employee.aggregate ([
    # lookup and match employees based on workplace_id
    {
        '$lookup': {
            'from': 'workplace',
            'localField': "workplace_id",
            'foreignField': "_id",
            'as': "Workplace_Details",
            }

    },
    {
        #count number of employees working for each workplace id
        '$group' : {
            '_id': '$Workplace_Details.name',
            'employeeCount':{'$sum':1}
        }
    },
    {
        #sort results in descending order
        '$sort': {'employeeCount':-1}
    },

    
])

pd.DataFrame(list(query_result))

,_id,employeeCount
0,[Hilton Solutions],15
1,[Health Net Inc.],14
2,[Aetna Inc.],13
3,[Bell Microproducts Inc.],11
4,[Equity Office Properties Trust],10
5,[Union Planters Corp],10
6,[Equity Residential Properties Trust],7
7,[Kemet Corp.],6
8,[Xcel Bear Inc],6
9,[Great Plains Energy Inc.],5
